# preprocessing (전처리)

- Data Cleansing : 결측치 이상치 제거 
- Data Encoding : 텍스트 데이터 > 숫자로 변환 (범주형 데이터)
- Data Scaling : 숫자값 정규화
- Outlier : 이상치 처리 
- Feature Engineering : 속성 생성 / 수정 / 가공

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

### Data Encoding

##### Label Encoder

    - 범주형 데이터에 대해 적절히 숫자로 변환하는 것

In [3]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '세탁기', 'PC', '전기난로', 'PC', 'TV', '믹서기', 'PC']

encoder = LabelEncoder()

encoder.fit(items)   # 전달받은 범주형 데이터를 기준으로, 중복값 제거 / 오름차순 정렬. > 적절한 숫자(인덱스숫자)에 맵핑
encoded_items = encoder.transform(items)
encoded_items

array([1, 2, 4, 0, 5, 0, 1, 3, 0])

In [4]:
encoder.classes_    

array(['PC', 'TV', '냉장고', '믹서기', '세탁기', '전기난로'], dtype='<U4')

##### One-hot Encoder
- 주어진 데이터를 희소배열로 변환 (One-vs-Rest 배열)
- 희소 배열이란 대부분이 0이고 특정 인덱스만 값을 가지고 있는 배열

In [5]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
# encoder.fit(items)    # 에러 발생. 2차원 배열을 받게 만들어짐. 

items_2d = np.array(items).reshape(-1, 1)
encoder = OneHotEncoder()
encoder.fit(items_2d) 
oh_items = encoder.transform(items_2d)
oh_items   # <Compressed Sparse Row sparse matrix of dtype 'float64' with 9 stored elements and shape (9, 6)> : 희소 행렬을 가짐
print(oh_items)

# <Compressed Sparse Row sparse matrix of dtype 'float64'
# 	with 9 stored elements and shape (9, 6)>
#   Coords	Values
#   (0, 1)	1.0     0번째 값은 1번 인덱스가 1이라는 내용 / 6개의 칼럼을 가지는 총 9개의 로우가 생김. 
#   (1, 2)	1.0
#   (2, 4)	1.0
#   (3, 0)	1.0
#   (4, 5)	1.0
#   (5, 0)	1.0
#   (6, 1)	1.0
#   (7, 3)	1.0
#   (8, 0)	1.0

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (9, 6)>
  Coords	Values
  (0, 1)	1.0
  (1, 2)	1.0
  (2, 4)	1.0
  (3, 0)	1.0
  (4, 5)	1.0
  (5, 0)	1.0
  (6, 1)	1.0
  (7, 3)	1.0
  (8, 0)	1.0


In [6]:
oh_items.toarray()   # 인코딩 형태 확인

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [7]:
encoder.categories_  

[array(['PC', 'TV', '냉장고', '믹서기', '세탁기', '전기난로'], dtype='<U4')]

In [8]:
# 라벨인코더의 경우, 숫자하나로 표현하기 때문에 간편하지만, 숫자 값으로 인해 특성 강화가(가중치) 일어날 수 있음.
# 원핫의 경우, 가중치가 부여되지는 않지만 메모리 할당이 많음. 
# 원핫인코더의 경우, array 형태로 되어있기 때문에 데이터 프레임 형태로 바꿔줘야함. 

- DataFrame에서 One-hot encoding

In [9]:
df = pd.DataFrame({
    'items' : ['TV', '냉장고', '세탁기', 'PC', '전기난로', 'PC', 'TV', '믹서기', 'PC']
})

df

,items
0,TV
1,냉장고
2,세탁기
3,PC
4,전기난로
5,PC
6,TV
7,믹서기
8,PC


In [10]:
df_dummies = pd.get_dummies(df, dtype=int)   # 기본값은 불리언(dtype 지정 필수)
df_dummies

df_dummies.to_numpy
np.array(df_dummies) # df_dummies.to_numpy와 동일 

array([[0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0]])

### Data Scaling (Feature Scalig)
- Scaling 작업은 train 데이터, test 데이터에 동일하게 적용해야함
    - fit() : train 데이터 
    - transform() : train 데이터, test 데이터 

In [11]:
from sklearn.datasets import load_iris

iris_ds = load_iris()

##### 표준정규화 (standardScaler)
- 평균이 0, 표준편차가 1인 값으로 변환
- 데이터가 정규분포인 경우, 적합
- 이상치에 덜 민감
- 선형회귀 및 로지스틱 회귀등의 알고리즘에 적합

In [12]:
from sklearn.preprocessing import StandardScaler

standard_sc = StandardScaler()
standard_sc.fit(iris_ds.data)
standard_sc.transform(iris_ds.data)

array([[-9.00681170e-01,  1.01900435e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00, -1.31979479e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.38535265e+00,  3.28414053e-01, -1.39706395e+00,
        -1.31544430e+00],
       [-1.50652052e+00,  9.82172869e-02, -1.28338910e+00,
        -1.31544430e+00],
       [-1.02184904e+00,  1.24920112e+00, -1.34022653e+00,
        -1.31544430e+00],
       [-5.37177559e-01,  1.93979142e+00, -1.16971425e+00,
        -1.05217993e+00],
       [-1.50652052e+00,  7.88807586e-01, -1.34022653e+00,
        -1.18381211e+00],
       [-1.02184904e+00,  7.88807586e-01, -1.28338910e+00,
        -1.31544430e+00],
       [-1.74885626e+00, -3.62176246e-01, -1.34022653e+00,
        -1.31544430e+00],
       [-1.14301691e+00,  9.82172869e-02, -1.28338910e+00,
        -1.44707648e+00],
       [-5.37177559e-01,  1.47939788e+00, -1.28338910e+00,
        -1.31544430e+00],
       [-1.26418478e+00,  7.88807586e-01, -1.22655167e+00,
      

##### 최소최대정규화 (MinMaxScaler)
- 0과 1사이의 값으로 데이터 변환
- 이상치에 민감하게 반응, 이상치가 있는 경우 데이터 왜곡 가능성
- SVM 및 KNN과 같은 거리 기반 모델에 적합 

In [13]:
from sklearn.preprocessing import MinMaxScaler

minmax_sc = MinMaxScaler()
# minmax_sc.fit_transform([[20], [30], [40]])   # 방법 확인하기 
minmax_sc.fit(iris_ds.data)
minmax_sc.transform(iris_ds.data)   

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

---

### 타이타닉 생존률 예측에 필요한 데이터 전처리 연습하기 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# [ 전처리 함수 ]
def fillna(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    """
    결측치 처리 함수
    - Age: 평균치로 대체
    - Cabin: 'N' 기본값으로 대체
    - Embarked: 'N' 기본값으로 대체
    """


def drop_feature(df):
    """
    모델 훈련과 관련 없는 속성 제거
    - PassengerId, Name, Ticket
    """
    return df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)


def encode_feature(df):
    """
    범주형 데이터를 숫자로 인코딩
    - Sex, Cabin, Embarked
    """
    encoder = LabelEncoder()
    encoder.fit(df['Sex'])  
    df['encoded_Sex'] = encoder.transform(df['Sex'])
    
    encoder.fit(df['Cabin'])  
    df['encoded_Cabin'] = encoder.transform(df['Cabin'])

    encoder.fit(df['Embarked'])  
    df['encoded_Embarked'] = encoder.transform(df['Embarked'])

    return df
    

def preprocess_data(df):
    """
    전처리 함수 호출
    """
    df = fillna(df)
    df = drop_feature(df)
    df = encode_feature(df)
    
    return df


def scailing_feature(train_data, test_data):
    """
    특성 스케일링 (정규화)
    """
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_data)
    test_scaled = scaler.transform(test_data)
    
    return train_scaled, test_scaled

In [ ]:
# 데이터 로드 
df = pd.read_csv('./data/titanic.csv')

In [16]:
# 전처리 


In [17]:
# 데이터 분리 


In [18]:
# 특성 스케일링


In [19]:
# LogisticRegression 모델 훈련
from sklearn.linear_model import LogisticRegression


In [20]:
# 평가 
